In [10]:
import rmsd
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d

import random


from mpl_toolkits.mplot3d.art3d import Line3DCollection
import plotly.graph_objects as go

In [13]:


def plot_tdg_file(path,chr,copy,color_column,plotly,size):
    df = pd.read_csv(path,sep="\t",comment='#')
    df.columns = ['chr','pos','x','y','z']
    df_chr = df.query("`chr` == @chr")
    print(f"len, {len(df_chr)}")
    pos = "pos"
    x = "x"
    y = "y"
    z = "z"
    title = f"{chr}-copy{copy}"
    if plotly == False:
        plotly_3d_chromosomes(df_chr, pos)
    else:
        plotly_3d_chromosomes(df_chr,x,y,z,pos,title,size)



def plot_3d_chromosomes(data, title,color_column
                , cmap_name='viridis'  # Changed to 'viridis' for better visibility
                , color_norm=(None, None)
                , figsize=(10, 6)
                , dpi=120
                , smoothness=500):
    """
    绘制带有颜色映射的染色体3D图。
    """

    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_subplot(111, projection='3d')

    # 设置颜色映射
    cmap = plt.cm.get_cmap(cmap_name)

    # 检查是否提供了颜色规范范围，如果没有，则根据数据计算
    if all(val is not None for val in color_norm):
        norm_min, norm_max = color_norm
    else:
        norm_min, norm_max = data[color_column].min(), data[color_column].max()

    norm = mcolors.Normalize(vmin=norm_min, vmax=norm_max)

    # 按染色体分组绘制
    for chrom, chrom_df in data.groupby('chrom'):
        x = chrom_df['X(nm)'].to_numpy()
        y = chrom_df['Y(nm)'].to_numpy()
        z = chrom_df['Z(nm)'].to_numpy()

        # 计算每个段的长度
        t = np.zeros(x.shape)
        t[1:] = np.sqrt((x[1:] - x[:-1])**2 + (y[1:] - y[:-1])**2 + (z[1:] - z[:-1])**2).cumsum()

        # 插值
        fx = interp1d(t, x, kind='cubic')
        fy = interp1d(t, y, kind='cubic')
        fz = interp1d(t, z, kind='cubic')

        t_smooth = np.linspace(0, t.max(), num=smoothness)
        x_smooth, y_smooth, z_smooth = fx(t_smooth), fy(t_smooth), fz(t_smooth)

        # 获取颜色
        color_values = cmap(norm(chrom_df[color_column].to_numpy()))

        # 准备线段和颜色
        points = np.array([x_smooth, y_smooth, z_smooth]).T.reshape(-1, 1, 3)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        line_colors = [color_values[int(np.floor(i * len(color_values) / len(t_smooth)))] for i in range(len(t_smooth) - 1)]

        # 绘制线条
        lc = Line3DCollection(segments, colors=line_colors)
        ax.add_collection3d(lc)

    # 颜色条和标签
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm, ax=ax, label=color_column)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(data['X(nm)'].min(), data['X(nm)'].max())
    ax.set_ylim(data['Y(nm)'].min(), data['Y(nm)'].max())
    ax.set_zlim(data['Z(nm)'].min(), data['Z(nm)'].max())
    ax.view_init(elev=30, azim=120)  # Set a nice angle
    #title
    ax.set_title(title)
    plt.show()

    
def plotly_3d_chromosomes(data, x_column, y_column, z_column, color_column, title,size):
    fig = go.Figure(data=go.Scatter3d(
        x=data[x_column], y=data[y_column], z=data[z_column],
        marker=dict(
            size=size,
            color=data[color_column],
            colorscale='Viridis',
        ),
        line=dict(
            color='darkblue',
            width=2
        )
    ))

    # 标题居中且确保有足够的空间显示标题
    fig.update_layout(
        title={
            'text': title,
            'y':0.9,  # 标题的y位置，根据需要调整
            'x':0.5,  # 标题的x位置，0.5表示居中
            'xanchor': 'center',  # 标题的x锚点
            'yanchor': 'top'  # 标题的y锚点
        },
        autosize=True,
        scene=dict(
            camera=dict(
                up=dict(x=0, y=0, z=1),
                eye=dict(x=0, y=1.0707, z=1),
            ),
            aspectratio=dict(x=1, y=1, z=0.7),
            aspectmode='manual'
        ),
        margin=dict(
            l=10,
            r=10,
            b=10,
            t=50  # 增加顶部边距以显示完整的标题
        ),
    )
    fig.show()


def plot_cell_file(chr,copy,x,y,z,pos,plotly):
    chrom = chr.split('chr')[1]
    chrom_path = f"/shareb/mliu/evaluate_impute/image_analysis/data/chromosome{chrom}.tsv"
    df_all = pd.read_csv(chrom_path,sep="\t")
    df_cell = df_all.query("`Chromosome copy number`== @copy")
    df_cell['chrom'] = df_cell['Genomic coordinate'].apply(lambda x: x.split(":")[0])
    df_cell['start'] = df_cell['Genomic coordinate'].apply(lambda x: x.split(":")[1].split("-")[0]).astype(int)
    df_cell['end'] = df_cell['Genomic coordinate'].apply(lambda x: x.split(":")[1].split("-")[1]).astype(int)
    df_cell['pos'] = ((df_cell['start'] + df_cell['end']) / 2).astype(int)
    df_cell.dropna(subset=x, inplace=True)
    df_cell.dropna(subset=y, inplace=True)
    df_cell.dropna(subset=z, inplace=True)
    data = df_cell
    smoothness=500
    title = f"{chr}-copy{copy}"
    if plotly == False:
        plotly_3d_chromosomes(data, pos)
    else:
        plotly_3d_chromosomes(data,x,y,z,pos,title)

In [4]:
filepath1 = "/share/Data/hxie/project/202209/esc_xwliu/esc1207/processed/d0A1/3d_info/50k.0.3dg"


In [8]:
df = pd.read_csv(filepath1,sep="\t",comment='#')
df.columns = ['chr','pos','x','y','z']
query_chr = f"{chr}a"
df_chr = df.query("`chr` == @query_chr")

In [15]:
tdg_path = filepath1
chr = "chrXa"
cell = "d0A1"
color_column = "pos"
plotly = True
size = 1.5
plot_tdg_file(tdg_path,chr,cell,color_column,plotly,size)

len, 3421


In [20]:
chr = "chrXb"
plot_tdg_file(tdg_path,chr,cell,color_column,plotly,size)

len, 3421


In [17]:
#整个的里面挑一个
filepath2 = "/share/Data/hxie/project/202209/esc_xwliu/esc1207/processed/d0A1/3d_info/clean.50k.0.3dg"
chr = "chr1(mat)"
plot_tdg_file(filepath2,chr,cell,color_column,plotly,size)

len, 3796


In [18]:
chr = "chr1(pat)"
plot_tdg_file(filepath2,chr,cell,color_column,plotly,size)

len, 3844


In [22]:
#整个的里面挑一个
filepath_xa = "/share/Data/hxie/project/202209/esc_xwliu/esc1207/processed/d0A1/3d_infoAllel/mat/50k.0.3dg"
chr = "chr1a"
plot_tdg_file(filepath_xa,chr,cell,color_column,plotly,size)

len, 3908
